# End-to-End Project: AI-Powered Portfolio Optimization

**Duration:** ~30 minutes  
**Goal:** Build an AI agent that performs real portfolio optimization using PyPortfolioOpt

---

## What We're Building

A portfolio optimization agent that can:
- Fetch real historical price data
- Calculate efficient frontiers
- Find optimal portfolios (Max Sharpe, Min Volatility)
- Suggest portfolio allocations
- Run what-if scenarios

This project brings together **everything** from the course:

| Pattern | How We Use It |
|---------|---------------|
| **Tool Calling** | Fetch prices, run optimizations |
| **ReAct** | Reason through optimization choices |
| **CodeAct** | Calculate metrics, generate reports |
| **Orchestration** | Combine all patterns seamlessly |
| **Memory** | Remember context for follow-ups |

---

## Part 1: Environment Setup

In [1]:
# Install required packages
!pip install -q smolagents yfinance pyportfolioopt pandas numpy matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.1/222.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 14.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.0.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.2 which is incompatible.


In [2]:
# Core imports
from smolagents import CodeAgent, tool
from smolagents import OpenAIServerModel
from smolagents.monitoring import LogLevel
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import getpass
import warnings
warnings.filterwarnings('ignore')

print("Imports ready!")

Imports ready!


In [4]:
# Configure API key
API_KEY = getpass.getpass("Enter your OpenAI API key: ")
model = OpenAIServerModel("gpt-4o-mini", api_key=API_KEY)
print("Model ready!")

Enter your OpenAI API key: ··········
Model ready!


---

## Part 2: Understanding PyPortfolioOpt

**PyPortfolioOpt** is a Python library for portfolio optimization based on Modern Portfolio Theory (MPT).

### Key Concepts

| Concept | Description |
|---------|-------------|
| **Efficient Frontier** | Set of portfolios offering highest return for each risk level |
| **Max Sharpe** | Portfolio with best risk-adjusted return |
| **Min Volatility** | Lowest-risk portfolio possible |
| **Expected Returns** | Predicted future returns (from historical data) |
| **Covariance Matrix** | How assets move together |

### The Optimization Flow

```
Historical Prices → Expected Returns → Covariance Matrix → Optimizer → Optimal Weights
```

In [5]:
# Quick PyPortfolioOpt demo (before we wrap it in tools)
from pypfopt import EfficientFrontier, expected_returns, risk_models

# Get sample data
tickers = ["AAPL", "MSFT", "AMZN"]
end_date = datetime.now()
start_date = end_date - timedelta(days=365*2)  # 2 years

# Download prices
prices = yf.download(tickers, start=start_date, end=end_date)["Close"]
print(f"Downloaded {len(prices)} days of data for {len(tickers)} stocks")
print(prices.tail())

[*********************100%***********************]  3 of 3 completed

Downloaded 501 days of data for 3 stocks
Ticker            AAPL        AMZN        MSFT
Date                                          
2026-02-11  275.500000  204.080002  404.369995
2026-02-12  261.730011  199.600006  401.839996
2026-02-13  255.779999  198.789993  401.320007
2026-02-17  263.880005  201.149994  396.859985
2026-02-18  264.649994  205.649994  400.790009


In [6]:
# Calculate expected returns and covariance
mu = expected_returns.mean_historical_return(prices)
S = risk_models.sample_cov(prices)

print("Expected Annual Returns:")
for ticker, ret in mu.items():
    print(f"  {ticker}: {ret*100:.1f}%")

print("\nCovariance Matrix (sample):")
print(S.round(4))

Expected Annual Returns:
  AAPL: 21.5%
  AMZN: 11.0%
  MSFT: 0.4%

Covariance Matrix (sample):
Ticker    AAPL    AMZN    MSFT
Ticker                        
AAPL    0.0808  0.0420  0.0284
AMZN    0.0420  0.0996  0.0436
MSFT    0.0284  0.0436  0.0572


In [7]:
# Find the Max Sharpe portfolio
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

print("Max Sharpe Portfolio Weights:")
for ticker, weight in cleaned_weights.items():
    if weight > 0:
        print(f"  {ticker}: {weight*100:.1f}%")

# Get performance metrics
performance = ef.portfolio_performance(verbose=True)

Max Sharpe Portfolio Weights:
  AAPL: 100.0%
Expected annual return: 21.5%
Annual volatility: 28.4%
Sharpe Ratio: 0.76


**That's PyPortfolioOpt in action!** Now let's wrap this in tools so our agent can use it.

---

## Part 3: Building the Tools

We'll create tools that give our agent access to:
1. Historical price data
2. Portfolio optimization
3. Performance analysis

In [8]:
# Tool 1: Fetch historical prices
@tool
def get_historical_prices(tickers: str, years: int = 2) -> str:
    """
    Fetch historical adjusted close prices for a list of stock tickers.

    Args:
        tickers: Comma-separated stock symbols (e.g., 'AAPL,MSFT,GOOGL')
        years: Number of years of historical data (default: 2)

    Returns:
        JSON string with price statistics and recent prices
    """
    import yfinance as yf
    import json
    from datetime import datetime, timedelta

    ticker_list = [t.strip().upper() for t in tickers.split(',')]
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365*years)

    try:
        prices = yf.download(ticker_list, start=start_date, end=end_date, progress=False)["Close"]

        if prices.empty:
            return json.dumps({"error": "No data found for tickers"})

        # Handle single ticker case
        if len(ticker_list) == 1:
            prices = prices.to_frame(name=ticker_list[0])

        # Calculate stats
        stats = {}
        for ticker in prices.columns:
            col = prices[ticker].dropna()
            if len(col) > 0:
                returns = col.pct_change().dropna()
                stats[ticker] = {
                    "latest_price": round(col.iloc[-1], 2),
                    "annual_return": round(returns.mean() * 252 * 100, 2),
                    "annual_volatility": round(returns.std() * np.sqrt(252) * 100, 2),
                    "data_points": len(col)
                }

        return json.dumps({
            "tickers": ticker_list,
            "period": f"{years} years",
            "statistics": stats,
            "data_ready": True
        }, indent=2)

    except Exception as e:
        return json.dumps({"error": str(e)})

print("Tool 1 created: get_historical_prices")

Tool 1 created: get_historical_prices


In [9]:
# Tool 2: Optimize portfolio for Max Sharpe Ratio
@tool
def optimize_max_sharpe(tickers: str, risk_free_rate: float = 0.02) -> str:
    """
    Find the portfolio with the maximum Sharpe ratio (best risk-adjusted return).

    Args:
        tickers: Comma-separated stock symbols (e.g., 'AAPL,MSFT,GOOGL,AMZN')
        risk_free_rate: Annual risk-free rate as decimal (default: 0.02 = 2%)

    Returns:
        JSON with optimal weights and expected performance
    """
    import yfinance as yf
    import json
    from datetime import datetime, timedelta
    from pypfopt import EfficientFrontier, expected_returns, risk_models

    ticker_list = [t.strip().upper() for t in tickers.split(',')]
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365*2)

    try:
        prices = yf.download(ticker_list, start=start_date, end=end_date, progress=False)["Close"]

        if len(ticker_list) == 1:
            return json.dumps({"error": "Need at least 2 tickers for optimization"})

        # Calculate returns and covariance
        mu = expected_returns.mean_historical_return(prices)
        S = risk_models.sample_cov(prices)

        # Optimize for max Sharpe
        ef = EfficientFrontier(mu, S)
        weights = ef.max_sharpe(risk_free_rate=risk_free_rate)
        cleaned_weights = ef.clean_weights()

        # Get performance
        expected_return, volatility, sharpe = ef.portfolio_performance(risk_free_rate=risk_free_rate)

        return json.dumps({
            "optimization": "Maximum Sharpe Ratio",
            "weights": {k: round(v*100, 2) for k, v in cleaned_weights.items()},
            "performance": {
                "expected_annual_return": round(expected_return*100, 2),
                "annual_volatility": round(volatility*100, 2),
                "sharpe_ratio": round(sharpe, 3)
            },
            "risk_free_rate": risk_free_rate
        }, indent=2)

    except Exception as e:
        return json.dumps({"error": str(e)})

print("Tool 2 created: optimize_max_sharpe")

Tool 2 created: optimize_max_sharpe


In [10]:
# Tool 3: Optimize for Minimum Volatility
@tool
def optimize_min_volatility(tickers: str) -> str:
    """
    Find the portfolio with minimum volatility (lowest risk).
    Best for conservative investors who prioritize capital preservation.

    Args:
        tickers: Comma-separated stock symbols (e.g., 'AAPL,MSFT,GOOGL,AMZN')

    Returns:
        JSON with optimal weights and expected performance
    """
    import yfinance as yf
    import json
    from datetime import datetime, timedelta
    from pypfopt import EfficientFrontier, expected_returns, risk_models

    ticker_list = [t.strip().upper() for t in tickers.split(',')]
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365*2)

    try:
        prices = yf.download(ticker_list, start=start_date, end=end_date, progress=False)["Close"]

        if len(ticker_list) == 1:
            return json.dumps({"error": "Need at least 2 tickers for optimization"})

        mu = expected_returns.mean_historical_return(prices)
        S = risk_models.sample_cov(prices)

        ef = EfficientFrontier(mu, S)
        weights = ef.min_volatility()
        cleaned_weights = ef.clean_weights()

        expected_return, volatility, sharpe = ef.portfolio_performance()

        return json.dumps({
            "optimization": "Minimum Volatility",
            "weights": {k: round(v*100, 2) for k, v in cleaned_weights.items()},
            "performance": {
                "expected_annual_return": round(expected_return*100, 2),
                "annual_volatility": round(volatility*100, 2),
                "sharpe_ratio": round(sharpe, 3)
            }
        }, indent=2)

    except Exception as e:
        return json.dumps({"error": str(e)})

print("Tool 3 created: optimize_min_volatility")

Tool 3 created: optimize_min_volatility


In [11]:
# Tool 4: Optimize for Target Return
@tool
def optimize_target_return(tickers: str, target_return: float) -> str:
    """
    Find the minimum volatility portfolio that achieves a specific target return.

    Args:
        tickers: Comma-separated stock symbols (e.g., 'AAPL,MSFT,GOOGL,AMZN')
        target_return: Desired annual return as decimal (e.g., 0.15 = 15%)

    Returns:
        JSON with optimal weights and expected performance
    """
    import yfinance as yf
    import json
    from datetime import datetime, timedelta
    from pypfopt import EfficientFrontier, expected_returns, risk_models

    ticker_list = [t.strip().upper() for t in tickers.split(',')]
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365*2)

    try:
        prices = yf.download(ticker_list, start=start_date, end=end_date, progress=False)["Close"]

        if len(ticker_list) == 1:
            return json.dumps({"error": "Need at least 2 tickers for optimization"})

        mu = expected_returns.mean_historical_return(prices)
        S = risk_models.sample_cov(prices)

        ef = EfficientFrontier(mu, S)

        try:
            weights = ef.efficient_return(target_return=target_return)
        except Exception:
            return json.dumps({"error": f"Target return {target_return*100}% may not be achievable with these assets"})

        cleaned_weights = ef.clean_weights()
        expected_return, volatility, sharpe = ef.portfolio_performance()

        return json.dumps({
            "optimization": f"Target Return ({target_return*100}%)",
            "weights": {k: round(v*100, 2) for k, v in cleaned_weights.items()},
            "performance": {
                "expected_annual_return": round(expected_return*100, 2),
                "annual_volatility": round(volatility*100, 2),
                "sharpe_ratio": round(sharpe, 3)
            }
        }, indent=2)

    except Exception as e:
        return json.dumps({"error": str(e)})

print("Tool 4 created: optimize_target_return")

Tool 4 created: optimize_target_return


In [12]:
# Tool 5: Calculate discrete allocation (actual shares to buy)
@tool
def calculate_allocation(tickers: str, total_investment: float, optimization: str = "max_sharpe") -> str:
    """
    Calculate the exact number of shares to buy for a given investment amount.

    Args:
        tickers: Comma-separated stock symbols (e.g., 'AAPL,MSFT,GOOGL,AMZN')
        total_investment: Total dollar amount to invest
        optimization: Strategy - 'max_sharpe' or 'min_volatility'

    Returns:
        JSON with shares to buy for each stock and leftover cash
    """
    import yfinance as yf
    import json
    from datetime import datetime, timedelta
    from pypfopt import EfficientFrontier, expected_returns, risk_models
    from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

    ticker_list = [t.strip().upper() for t in tickers.split(',')]
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365*2)

    try:
        prices = yf.download(ticker_list, start=start_date, end=end_date, progress=False)["Close"]

        if len(ticker_list) == 1:
            return json.dumps({"error": "Need at least 2 tickers for optimization"})

        mu = expected_returns.mean_historical_return(prices)
        S = risk_models.sample_cov(prices)

        ef = EfficientFrontier(mu, S)

        if optimization == "min_volatility":
            weights = ef.min_volatility()
        else:
            weights = ef.max_sharpe()

        cleaned_weights = ef.clean_weights()
        latest_prices = get_latest_prices(prices)

        da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=total_investment)
        allocation, leftover = da.greedy_portfolio()

        # Calculate actual investment per stock
        investment_details = {}
        for ticker, shares in allocation.items():
            price = latest_prices[ticker]
            investment_details[ticker] = {
                "shares": shares,
                "price_per_share": round(price, 2),
                "total_investment": round(shares * price, 2)
            }

        return json.dumps({
            "optimization": optimization,
            "total_investment": total_investment,
            "allocation": investment_details,
            "leftover_cash": round(leftover, 2),
            "invested_amount": round(total_investment - leftover, 2)
        }, indent=2)

    except Exception as e:
        return json.dumps({"error": str(e)})

print("Tool 5 created: calculate_allocation")

Tool 5 created: calculate_allocation


In [13]:
# Tool 6: Compare optimization strategies
@tool
def compare_strategies(tickers: str) -> str:
    """
    Compare Max Sharpe vs Min Volatility strategies for the same assets.

    Args:
        tickers: Comma-separated stock symbols (e.g., 'AAPL,MSFT,GOOGL,AMZN')

    Returns:
        JSON comparing both strategies side-by-side
    """
    import yfinance as yf
    import json
    from datetime import datetime, timedelta
    from pypfopt import EfficientFrontier, expected_returns, risk_models

    ticker_list = [t.strip().upper() for t in tickers.split(',')]
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365*2)

    try:
        prices = yf.download(ticker_list, start=start_date, end=end_date, progress=False)["Close"]

        if len(ticker_list) == 1:
            return json.dumps({"error": "Need at least 2 tickers for optimization"})

        mu = expected_returns.mean_historical_return(prices)
        S = risk_models.sample_cov(prices)

        # Max Sharpe
        ef_sharpe = EfficientFrontier(mu, S)
        ef_sharpe.max_sharpe()
        sharpe_weights = ef_sharpe.clean_weights()
        sharpe_perf = ef_sharpe.portfolio_performance()

        # Min Volatility
        ef_vol = EfficientFrontier(mu, S)
        ef_vol.min_volatility()
        vol_weights = ef_vol.clean_weights()
        vol_perf = ef_vol.portfolio_performance()

        return json.dumps({
            "comparison": {
                "max_sharpe": {
                    "weights": {k: round(v*100, 2) for k, v in sharpe_weights.items()},
                    "expected_return": round(sharpe_perf[0]*100, 2),
                    "volatility": round(sharpe_perf[1]*100, 2),
                    "sharpe_ratio": round(sharpe_perf[2], 3)
                },
                "min_volatility": {
                    "weights": {k: round(v*100, 2) for k, v in vol_weights.items()},
                    "expected_return": round(vol_perf[0]*100, 2),
                    "volatility": round(vol_perf[1]*100, 2),
                    "sharpe_ratio": round(vol_perf[2], 3)
                }
            },
            "insight": "Max Sharpe maximizes risk-adjusted returns; Min Volatility minimizes risk."
        }, indent=2)

    except Exception as e:
        return json.dumps({"error": str(e)})

print("Tool 6 created: compare_strategies")

Tool 6 created: compare_strategies


In [14]:
# Test the tools
print("Testing get_historical_prices:")
print(get_historical_prices("AAPL,MSFT,GOOGL"))

Testing get_historical_prices:
{
  "tickers": [
    "AAPL",
    "MSFT",
    "GOOGL"
  ],
  "period": "2 years",
  "statistics": {
    "AAPL": {
      "latest_price": 264.62,
      "annual_return": 23.44,
      "annual_volatility": 28.43,
      "data_points": 501
    },
    "GOOGL": {
      "latest_price": 303.62,
      "annual_return": 43.46,
      "annual_volatility": 29.9,
      "data_points": 501
    },
    "MSFT": {
      "latest_price": 400.8,
      "annual_return": 3.26,
      "annual_volatility": 23.91,
      "data_points": 501
    }
  },
  "data_ready": true
}


In [15]:
print("\nTesting optimize_max_sharpe:")
print(optimize_max_sharpe("AAPL,MSFT,AMZN"))


Testing optimize_max_sharpe:
{
  "optimization": "Maximum Sharpe Ratio",
  "weights": {
    "AAPL": 100.0,
    "AMZN": 0.0,
    "MSFT": 0.0
  },
  "performance": {
    "expected_annual_return": 21.46,
    "annual_volatility": 28.43,
    "sharpe_ratio": 0.685
  },
  "risk_free_rate": 0.02
}


---

## Part 4: Creating the Portfolio Optimization Agent

Now we combine all tools into an intelligent agent that can reason through portfolio decisions.

In [16]:
# Create the portfolio optimization agent
portfolio_optimizer = CodeAgent(
    tools=[
        get_historical_prices,
        optimize_max_sharpe,
        optimize_min_volatility,
        optimize_target_return,
        calculate_allocation,
        compare_strategies
    ],
    model=model,
    verbosity_level=LogLevel.INFO,
    max_steps=10,
    additional_authorized_imports=["numpy", "pandas", "json"]
)

print("Portfolio Optimization Agent ready!")
print("\nAvailable tools:")
print("  - get_historical_prices: Fetch price data and statistics")
print("  - optimize_max_sharpe: Find best risk-adjusted portfolio")
print("  - optimize_min_volatility: Find lowest-risk portfolio")
print("  - optimize_target_return: Find portfolio for specific return goal")
print("  - calculate_allocation: Convert weights to actual shares")
print("  - compare_strategies: Side-by-side strategy comparison")

Portfolio Optimization Agent ready!

Available tools:
  - get_historical_prices: Fetch price data and statistics
  - optimize_max_sharpe: Find best risk-adjusted portfolio
  - optimize_min_volatility: Find lowest-risk portfolio
  - optimize_target_return: Find portfolio for specific return goal
  - calculate_allocation: Convert weights to actual shares
  - compare_strategies: Side-by-side strategy comparison


---

## Part 5: Output Formatting Helper

Agent outputs can be raw JSON or dictionaries. Let's create a helper function that uses a second LLM call to format results into clean, human-readable summaries.

In [17]:
def format_response(raw_result, context: str = "") -> str:
    """
    Use an LLM to format raw agent output into a clean, human-readable summary.

    Args:
        raw_result: The raw output from the agent (dict, JSON string, or text)
        context: Optional context about what the query was asking for

    Returns:
        A nicely formatted string summary
    """
    from openai import OpenAI

    client = OpenAI(api_key=API_KEY)

    prompt = f"""Format the following portfolio analysis result into a clear, human-readable summary.

Use this structure:
- Start with a brief headline summary (1 sentence)
- Use bullet points for allocations and metrics
- Include any recommendations or insights
- Keep it concise but complete

{f"Context: {context}" if context else ""}

Raw result:
{raw_result}

Format this as a clean, professional summary:"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=500
    )

    return response.choices[0].message.content

print("format_response() helper created!")

format_response() helper created!


### The Two-Stage Pattern

```
┌─────────────────────────────────────────────────────────────┐
│                    OUTPUT FORMATTING                         │
│                                                              │
│  User Query → Agent → Raw Output → LLM Formatter → Summary  │
│                                                              │
│  Stage 1: Agent does the heavy lifting (tools, reasoning)   │
│  Stage 2: LLM formats raw data into readable text           │
└─────────────────────────────────────────────────────────────┘
```

**Why this pattern?**
- Agent outputs are often JSON/dicts (machine-readable)
- Users want natural language summaries (human-readable)
- A second LLM call is cheap and fast for formatting

---

## Part 6: Using the Agent with Formatted Output

In [18]:
# Query 1: Basic portfolio optimization with formatted output
raw_result = portfolio_optimizer.run("""
I want to build a tech-focused portfolio with these stocks:
AAPL, MSFT, GOOGL, NVDA, META

Find the optimal allocation using the Max Sharpe strategy.
Show me the expected performance metrics.
""")

# Format the raw output into a clean summary
print("\n" + "="*50)
print("FORMATTED SUMMARY:")
print("="*50)
formatted = format_response(raw_result, "Tech portfolio optimization using Max Sharpe")
print(formatted)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ I want to build a tech-focused portfolio with these stocks:                                                     │
│ AAPL, MSFT, GOOGL, NVDA, META                                                                                   │
│                                                                                                                 │
│ Find the optimal allocation using the Max Sharpe strategy.                                                      │
│ Show me the expected performance metrics.                                                                       │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  tickers = "AAPL,MSFT,GOOGL,NVDA,META"                                                                            
  optimization_result = optimize_max_sharpe(tickers=tickers, risk_free_rate=0.02)                                  
  print(optimization_result)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
{
  "optimization": "Maximum Sharpe Ratio",
  "weights": {
    "AAPL": 0.0,
    "GOOGL": 72.82,
    "META": 0.0,
    "MSFT": 0.0,
    "NVDA": 27.18
  },
  "performance": {
    "expected_annual_return": 52.65,
    "annual_volatility": 29.99,
    "sharpe_ratio": 1.689
  },
  "risk_free_rate": 0.02
}

Out: None

[Step 1: Duration 5.04 seconds| Input tokens: 2,504 | Output tokens: 155]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_result = {                                                                                                 
      "optimal_weights": {                                                                                         
          "GOOGL": 72.82,                                                                                          
          "NVDA": 27.18,                                                                                           
          "AAPL": 0,                                                                                               
          "META": 0,                                                                                               
          "MSFT": 0                                                                                                
      },                                                                                                           
      "performance_metrics": {                                                                                     
          "expected_annual_return": 52.65,                                                                         
          "annual_volatility": 29.99,                                                                              
          "sharpe_ratio": 1.689                                                                                    
      }                                                                                                            
  }                                                                                                                
                                                                                                                   
  final_answer(final_result)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: {'optimal_weights': {'GOOGL': 72.82, 'NVDA': 27.18, 'AAPL': 0, 'META': 0, 'MSFT': 0}, 
'performance_metrics': {'expected_annual_return': 52.65, 'annual_volatility': 29.99, 'sharpe_ratio': 1.689}}

[Step 2: Duration 4.45 seconds| Input tokens: 5,387 | Output tokens: 392]


FORMATTED SUMMARY:
**Headline Summary:** The optimized tech portfolio prioritizes Google and NVIDIA, achieving a high expected return with a favorable Sharpe ratio.

- **Portfolio Allocations:**
  - Google (GOOGL): 72.82%
  - NVIDIA (NVDA): 27.18%
  - Apple (AAPL): 0%
  - Meta (META): 0%
  - Microsoft (MSFT): 0%

- **Performance Metrics:**
  - Expected Annual Return: 52.65%
  - Annual Volatility: 29.99%
  - Sharpe Ratio: 1.689

**Recommendations and Insights:**
- Consider diversifying further into other tech stocks to mitigate risk, as the current allocation heavily favors Google and NVIDIA.
- Monitor market conditions that may affect the volatility of the portfolio, given the high annual volatility figure.


In [19]:
# Query 2: Compare strategies (uses memory from previous query)
result = portfolio_optimizer.run("""
Compare the Max Sharpe strategy with a Min Volatility approach
for the same tech stocks. Which would you recommend for a
conservative investor nearing retirement?
""", reset=False)

print("\n" + "="*50)
print("RESULT:")
print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Compare the Max Sharpe strategy with a Min Volatility approach                                                  │
│ for the same tech stocks. Which would you recommend for a                                                       │
│ conservative investor nearing retirement?                                                                       │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  tickers = "AAPL,MSFT,GOOGL,NVDA,META"                                                                            
  min_volatility_result = optimize_min_volatility(tickers=tickers)                                                 
  print(min_volatility_result)                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
{
  "optimization": "Minimum Volatility",
  "weights": {
    "AAPL": 26.39,
    "GOOGL": 20.35,
    "META": 0.92,
    "MSFT": 52.34,
    "NVDA": 0.0
  },
  "performance": {
    "expected_annual_return": 15.73,
    "annual_volatility": 20.97,
    "sharpe_ratio": 0.75
  }
}

Out: None

[Step 3: Duration 3.59 seconds| Input tokens: 8,794 | Output tokens: 533]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  comparison_result = {                                                                                            
      "Max_Sharpe": {                                                                                              
          "optimal_weights": {                                                                                     
              "GOOGL": 72.82,                                                                                      
              "NVDA": 27.18,                                                                                       
              "AAPL": 0,                                                                                           
              "META": 0,                                                                                           
              "MSFT": 0                                                                                            
          },                                                                                                       
          "performance_metrics": {                                                                                 
              "expected_annual_return": 52.65,                                                                     
              "annual_volatility": 29.99,                                                                          
              "sharpe_ratio": 1.689                                                                                
          }                                                                                                        
      },                                                                                                           
      "Min_Volatility": {                                                                                          
          "optimal_weights": {                                                                                     
              "AAPL": 26.39,                                                                                       
              "GOOGL": 20.35,                                                                                      
              "META": 0.92,                                                                                        
              "MSFT": 52.34,                                                                                       
              "NVDA": 0                                                                                            
          },                                                                                                       
          "performance_metrics": {                                                                                 
              "expected_annual_return": 15.73,                                                                     
              "annual_volatility": 20.97,                                                                          
              "sharpe_ratio": 0.75                                                                                 
          }                                                                                                        
      },                                                                                                           
      "recommendation": "For a conservative investor nearing retirement, Min Volatility is recommended due to its  
  lower risk profile and more stable returns."                                                                     
  }                                                                                                                
                                                                                                                   
  final_answer(comparison_result)                        

Final answer: {'Max_Sharpe': {'optimal_weights': {'GOOGL': 72.82, 'NVDA': 27.18, 'AAPL': 0, 'META': 0, 'MSFT': 0}, 
'performance_metrics': {'expected_annual_return': 52.65, 'annual_volatility': 29.99, 'sharpe_ratio': 1.689}}, 
'Min_Volatility': {'optimal_weights': {'AAPL': 26.39, 'GOOGL': 20.35, 'META': 0.92, 'MSFT': 52.34, 'NVDA': 0}, 
'performance_metrics': {'expected_annual_return': 15.73, 'annual_volatility': 20.97, 'sharpe_ratio': 0.75}}, 
'recommendation': 'For a conservative investor nearing retirement, Min Volatility is recommended due to its lower 
risk profile and more stable returns.'}

[Step 4: Duration 13.29 seconds| Input tokens: 12,548 | Output tokens: 1,141]


RESULT:
{'Max_Sharpe': {'optimal_weights': {'GOOGL': 72.82, 'NVDA': 27.18, 'AAPL': 0, 'META': 0, 'MSFT': 0}, 'performance_metrics': {'expected_annual_return': 52.65, 'annual_volatility': 29.99, 'sharpe_ratio': 1.689}}, 'Min_Volatility': {'optimal_weights': {'AAPL': 26.39, 'GOOGL': 20.35, 'META': 0.92, 'MSFT': 52.34, 'NVDA': 0}, 'performance_metrics': {'expected_annual_return': 15.73, 'annual_volatility': 20.97, 'sharpe_ratio': 0.75}}, 'recommendation': 'For a conservative investor nearing retirement, Min Volatility is recommended due to its lower risk profile and more stable returns.'}


In [20]:
# Query 3: Discrete allocation with formatted output
raw_result = portfolio_optimizer.run("""
I have $50,000 to invest. Using the Max Sharpe strategy for
AAPL, MSFT, GOOGL, NVDA, META - tell me exactly how many shares
of each stock I should buy.
""")

# Format the output
print("\n" + "="*50)
print("FORMATTED SUMMARY:")
print("="*50)
formatted = format_response(raw_result, "$50,000 investment allocation")
print(formatted)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ I have $50,000 to invest. Using the Max Sharpe strategy for                                                     │
│ AAPL, MSFT, GOOGL, NVDA, META - tell me exactly how many shares                                                 │
│ of each stock I should buy.                                                                                     │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  optimal_allocation = optimize_max_sharpe(tickers="AAPL,MSFT,GOOGL,NVDA,META", risk_free_rate=0.02)               
  print("Optimal allocation weights:", optimal_allocation)                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Optimal allocation weights: {
  "optimization": "Maximum Sharpe Ratio",
  "weights": {
    "AAPL": 0.0,
    "GOOGL": 72.71,
    "META": 0.0,
    "MSFT": 0.0,
    "NVDA": 27.29
  },
  "performance": {
    "expected_annual_return": 52.6,
    "annual_volatility": 30.0,
    "sharpe_ratio": 1.687
  },
  "risk_free_rate": 0.02
}

Out: None

[Step 1: Duration 4.15 seconds| Input tokens: 2,505 | Output tokens: 170]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  goog_shares_final = round(goog_shares)                                                                           
  nvda_shares_final = round(nvda_shares)                                                                           
                                                                                                                   
  final_answer({"GOOGL": goog_shares_final, "NVDA": nvda_shares_final})                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: {'GOOGL': 120, 'NVDA': 72}

[Step 5: Duration 2.97 seconds| Input tokens: 16,451 | Output tokens: 903]


FORMATTED SUMMARY:
**Headline Summary:** The portfolio consists of a strategic allocation in leading technology stocks, GOOGL and NVDA, totaling $50,000.

- **Investment Allocation:**
  - GOOGL: $30,000 (120 shares)
  - NVDA: $20,000 (72 shares)

- **Key Metrics:**
  - Total Investment: $50,000
  - GOOGL Share Price: $250 (assumed for calculation)
  - NVDA Share Price: $277.78 (assumed for calculation)

**Recommendations and Insights:**
- Consider diversifying into other sectors to mitigate risk.
- Monitor market trends for both companies to optimize timing for potential rebalancing.


In [21]:
# Query 4: Target return optimization
result = portfolio_optimizer.run("""
I need a portfolio that targets 20% annual return using
AAPL, MSFT, AMZN, GOOGL, NVDA.

Is this achievable? What's the risk involved?
""")

print("\n" + "="*50)
print("RESULT:")
print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ I need a portfolio that targets 20% annual return using                                                         │
│ AAPL, MSFT, AMZN, GOOGL, NVDA.                                                                                  │
│                                                                                                                 │
│ Is this achievable? What's the risk involved?                                                                   │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  target_return = 0.20                                                                                             
  tickers = "AAPL,MSFT,AMZN,GOOGL,NVDA"                                                                            
  portfolio_result = optimize_target_return(tickers=tickers, target_return=target_return)                          
  print(portfolio_result)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
{
  "optimization": "Target Return (20.0%)",
  "weights": {
    "AAPL": 27.07,
    "AMZN": 0.0,
    "GOOGL": 29.49,
    "MSFT": 43.44,
    "NVDA": 0.0
  },
  "performance": {
    "expected_annual_return": 20.0,
    "annual_volatility": 21.14,
    "sharpe_ratio": 0.946
  }
}

Out: None

[Step 1: Duration 4.35 seconds| Input tokens: 2,497 | Output tokens: 175]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "achievable": True,                                                                                          
      "portfolio_weights": {                                                                                       
          "AAPL": 27.07,                                                                                           
          "MSFT": 43.44,                                                                                           
          "GOOGL": 29.49,                                                                                          
          "AMZN": 0.0,                                                                                             
          "NVDA": 0.0                                                                                              
      },                                                                                                           
      "expected_annual_return": 20.0,                                                                              
      "annual_volatility": 21.14,                                                                                  
      "sharpe_ratio": 0.946                                                                                        
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: {'achievable': True, 'portfolio_weights': {'AAPL': 27.07, 'MSFT': 43.44, 'GOOGL': 29.49, 'AMZN': 0.0,
'NVDA': 0.0}, 'expected_annual_return': 20.0, 'annual_volatility': 21.14, 'sharpe_ratio': 0.946}

[Step 2: Duration 6.09 seconds| Input tokens: 5,391 | Output tokens: 470]


RESULT:
{'achievable': True, 'portfolio_weights': {'AAPL': 27.07, 'MSFT': 43.44, 'GOOGL': 29.49, 'AMZN': 0.0, 'NVDA': 0.0}, 'expected_annual_return': 20.0, 'annual_volatility': 21.14, 'sharpe_ratio': 0.946}


---

## Part 6: Advanced Analysis with CodeAct

The agent can also write custom code for more complex analysis.

In [22]:
# Query 5: Complex analysis requiring CodeAct
result = portfolio_optimizer.run("""
Analyze the diversification benefit of this portfolio:
AAPL, MSFT, GOOGL, JPM, JNJ, XOM

1. First get the historical data for all stocks
2. Run both Max Sharpe and Min Volatility optimizations
3. Calculate how much risk reduction we get from diversification
4. Which sectors should we add more of to improve diversification?

Provide a comprehensive analysis report.
""")

print("\n" + "="*50)
print("RESULT:")
print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Analyze the diversification benefit of this portfolio:                                                          │
│ AAPL, MSFT, GOOGL, JPM, JNJ, XOM                                                                                │
│                                                                                                                 │
│ 1. First get the historical data for all stocks                                                                 │
│ 2. Run both Max Sharpe and Min Volatility optimizations                                                         │
│ 3. Calculate how much risk reduction we get from diversification                                                │
│ 4. Which sectors should we add more of to improve diversification?                                              │
│                                                                                                                 │
│ Provide a comprehensive analysis report.                                                                        │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  historical_data = get_historical_prices(tickers='AAPL,MSFT,GOOGL,JPM,JNJ,XOM', years=2)                          
  print(historical_data)                                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
{
  "tickers": [
    "AAPL",
    "MSFT",
    "GOOGL",
    "JPM",
    "JNJ",
    "XOM"
  ],
  "period": "2 years",
  "statistics": {
    "AAPL": {
      "latest_price": 264.75,
      "annual_return": 23.47,
      "annual_volatility": 28.43,
      "data_points": 501
    },
    "GOOGL": {
      "latest_price": 302.89,
      "annual_return": 43.34,
      "annual_volatility": 29.9,
      "data_points": 501
    },
    "JNJ": {
      "latest_price": 244.38,
      "annual_return": 26.32,
      "annual_volatility": 17.75,
      "data_points": 501
    },
    "JPM": {
      "latest_price": 311.16,
      "annual_return": 32.96,
      "annual_volatility": 25.0,
      "data_points": 501
    },
    "MSFT": {
      "latest_price": 400.0,
      "annual_return": 3.16,
      "annual_volatility": 23.91,
      "data_points": 501
    },
    "XOM": {
      "latest_price": 149.16,
      "annual_return": 24.64,
      "annual_volatility": 22.19,
      "data_points": 501
    }
  },
  "data_ready": true
}

Out: None

[Step 1: Duration 6.86 seconds| Input tokens: 2,541 | Output tokens: 183]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 6 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_report = {                                                                                                 
      "original_portfolio_volatility": original_portfolio_volatility,                                              
      "optimized_portfolio_volatility": optimized_volatility,                                                      
      "risk_reduction": risk_reduction,                                                                            
      "suggested_additional_sectors": ["Consumer Discretionary", "Utilities", "Real Estate"]                       
  }                                                                                                                
                                                                                                                   
  final_answer(final_report)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: {'original_portfolio_volatility': 24.529999999999998, 'optimized_portfolio_volatility': 11.77, 
'risk_reduction': 12.759999999999998, 'suggested_additional_sectors': ['Consumer Discretionary', 'Utilities', 'Real
Estate']}

[Step 6: Duration 5.49 seconds| Input tokens: 23,448 | Output tokens: 1,514]


RESULT:
{'original_portfolio_volatility': 24.529999999999998, 'optimized_portfolio_volatility': 11.77, 'risk_reduction': 12.759999999999998, 'suggested_additional_sectors': ['Consumer Discretionary', 'Utilities', 'Real Estate']}


---

## Part 7: What You've Built

### The Complete System

```
┌─────────────────────────────────────────────────────────────────┐
│              AI-POWERED PORTFOLIO OPTIMIZATION                   │
│                                                                  │
│  ┌─────────────┐    ┌──────────────────┐    ┌───────────────┐  │
│  │   USER      │───►│  SMOLAGENTS      │───►│ PYPORTFOLIOOPT│  │
│  │   QUERY     │    │  AGENT           │    │ + YFINANCE    │  │
│  └─────────────┘    └──────────────────┘    └───────────────┘  │
│                              │                                   │
│                              ▼                                   │
│                     ┌──────────────────┐                        │
│                     │  PATTERNS USED:  │                        │
│                     │  - Tool Calling  │                        │
│                     │  - ReAct         │                        │
│                     │  - CodeAct       │                        │
│                     │  - Memory        │                        │
│                     └──────────────────┘                        │
│                              │                                   │
│                              ▼                                   │
│                     ┌──────────────────┐                        │
│                     │  DELIVERABLES:   │                        │
│                     │  - Optimal weights│                       │
│                     │  - Share counts  │                        │
│                     │  - Risk metrics  │                        │
│                     │  - Recommendations│                       │
│                     └──────────────────┘                        │
└─────────────────────────────────────────────────────────────────┘
```

### Key Concepts Demonstrated

| Concept | Implementation |
|---------|----------------|
| **Modern Portfolio Theory** | Efficient frontier, Sharpe ratio |
| **Tool Calling** | 6 specialized optimization tools |
| **ReAct Reasoning** | Agent decides which tool to use |
| **CodeAct** | Custom analysis and calculations |
| **Memory** | Follow-up questions with context |
| **Real Data** | yfinance for live market prices |

---

## Your Turn: Challenges

Try these challenges to extend what you've built:

### Challenge 1: Sector Diversification
Build a portfolio with stocks from 5 different sectors. Compare the risk profile to a tech-only portfolio.

### Challenge 2: Rebalancing Scenario
Ask the agent to compare your current portfolio allocation to the optimal one and suggest trades to rebalance.

### Challenge 3: Risk Budget
Tell the agent your maximum acceptable annual volatility (e.g., 15%) and ask it to find the highest-return portfolio within that constraint.

### Challenge 4: Add a New Tool
Create a tool that calculates Value at Risk (VaR) for the optimized portfolio.

In [ ]:
# CHALLENGE WORKSPACE
# Try your own queries here!

result = portfolio_optimizer.run("""
    YOUR CHALLENGE QUERY HERE
""")

---

## Recap: What You Learned

In this end-to-end project, you:

1. **Learned PyPortfolioOpt** — Mean-variance optimization, efficient frontier, Sharpe ratio

2. **Built 6 Tools** — Each wrapping a specific optimization capability

3. **Created an Agent** — That reasons through portfolio decisions autonomously

4. **Applied All 4 Patterns:**
   - Tool Calling: Fetch data and run optimizations
   - ReAct: Reason about which strategy fits the user's needs
   - CodeAct: Calculate custom metrics and format reports
   - Memory: Handle follow-up questions with context

5. **Used Real Data** — Live market prices via yfinance

---

**You've gone from theory to a working portfolio optimization system.**

This is the foundation for building production-grade financial AI applications.